<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/phi4_r1_grpo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers vllm accelerate

In [1]:
def halu_format_func(reference, query, response):
    system = """Your job is to evaluate whether a machine learning model has hallucinated or not. A hallucination occurs when the response is coherent but factually incorrect or nonsensical outputs that are not grounded in the provided context.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

    prompt = f"""You are given the following information:
        <info>
        [Knowledge]: {reference}
        [User Input]: {query}
        [Model Response]: {response}
        </info>
        Based on the information provided is the model output a hallucination?
        """
    return system, prompt

def rag_format_func(reference, query):
    system = """Your job is to evaluate whether a retrieved context is relevant, or unrelated to a user query.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

    prompt = f"""You are comparing a reference text to a question and trying to determine if the reference text
        contains information relevant to answering the question. Here is the info:
        <info>
        [Question]: {query}
        [Reference text]: {reference}
        </info>
        Compare the Question above to the Reference text. You must determine whether the Reference text
        contains information that can answer the Question. Please focus on whether the very specific
        question can be answered by the information in the Reference text.
        Your response must be single word, either "relevant" or "unrelated",
        and should not contain any text or characters aside from that word.
        "unrelated" means that the reference text does not contain an answer to the Question.
        "relevant" means the reference text contains an answer to the Question.
        Based on the information provided, is the provided reference relevant or unrelated to the question?"""
    return system, prompt

def toxic_format_func(text):
    system = """Your job is to evaluate whether a human text is toxic or non-toxic.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

    prompt = f"""You are examining written text content. Here is the text:
        <info>
        [Text]: {text}
        </info>
        Examine the text and determine whether the text is toxic or not. Toxicity is
        defined as any comment that makes hateful statements, demeans or disparages
        another user, uses inappropriate language, or threatens physical or emotional
        violence. Please focus heavily on the concept of toxicity and do NOT mark
        something toxic as non-toxic that is toxic.
        "toxic" means that the text meets the definition of toxic. "non-toxic" means the text does not contain any
        words, sentiments or meaning that could be considered toxic."""
    return system, prompt

In [2]:
from vllm import LLM, SamplingParams

# Configure sampling parameters
sampling_params = SamplingParams(
    temperature=0.1,
    top_p=0.1,
    max_tokens=1024,
)

# Initialize the LLM
llm = LLM(
    model="microsoft/phi-4", #"Jlonge4/phi4-r1-merge",  #microsoft/phi-4
    max_num_seqs=1,
    max_model_len=2048,
    tensor_parallel_size=1,  # Increase for multi-GPU inference
    gpu_memory_utilization=0.9,
)

INFO 02-15 22:08:19 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

INFO 02-15 22:08:34 config.py:542] This model supports multiple tasks: {'classify', 'score', 'embed', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 02-15 22:08:34 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=microsoft/phi-4, num_scheduler_steps=1, multi_step_str

tokenizer_config.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 02-15 22:10:56 model_runner.py:1115] Loading model weights took 27.3876 GB
INFO 02-15 22:10:57 worker.py:267] Memory profiling takes 1.21 seconds
INFO 02-15 22:10:57 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.90) = 35.60GiB
INFO 02-15 22:10:57 worker.py:267] model weights take 27.39GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 0.27GiB; the rest of the memory reserved for KV Cache is 7.85GiB.
INFO 02-15 22:10:58 executor_base.py:110] # CUDA blocks: 2572, # CPU blocks: 1310
INFO 02-15 22:10:58 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 20.09x
INFO 02-15 22:11:01 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 1/1 [00:00<00:00,  1.24it/s]

INFO 02-15 22:11:01 model_runner.py:1562] Graph capturing finished in 1 secs, took 0.05 GiB
INFO 02-15 22:11:01 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 5.40 seconds


In [3]:
from transformers import AutoTokenizer

def run_inference(system, prompt):
  tokenizer = AutoTokenizer.from_pretrained("Jlonge4/phi4-r1-guard")

  # Define prompts
  text = tokenizer.apply_chat_template([
      {"role" : "system", "content" : system},
      {"role" : "user", "content" : prompt},
    ], tokenize = False, add_generation_prompt = True)

  prompts = [
      text
  ]

  # Generate responses
  outputs = llm.generate(prompts, sampling_params)

  # Print results
  for output in outputs:
      prompt = output.prompt
      generated_text = output.outputs[0].text
      # print(f"Prompt: {prompt}")
      # print('------------------'*40)
      # print(f"Generated text: {generated_text}\n")
  return generated_text

# Test GRPO model

In [12]:
# RAG Relevance Examples
relevant_test = rag_format_func(
    reference="The process of photosynthesis in plants involves two main stages: light-dependent reactions and light-independent reactions (Calvin cycle). During the light-independent reactions, carbon dioxide is converted into glucose using the energy and compounds produced during the light-dependent reactions.",
    query="what happens in the light independent reactions of photosynthesis?"
)

unrelated_test = rag_format_func(
    reference="The Great Wall of China is an ancient series of walls and fortifications located in northern China, built around 500 years ago. The Great Wall is over 13,000 miles long.",
    query="what happens in the light independent reactions of photosynthesis?"
)

# Toxicity Examples
toxic_test = toxic_format_func(
    text="You're such an idiot, I hope you die in a fire. Nobody likes you and you should just disappear forever."
)

non_toxic_test = toxic_format_func(
    text="I disagree with your perspective on this issue, but I appreciate you sharing your thoughts. Perhaps we could discuss this further to better understand each other's viewpoints."
)

# Hallucination Examples
hallucination_yes = halu_format_func(
    reference="The Eiffel Tower was completed in 1889 and stands 324 meters tall. It was built for the World's Fair in Paris.",
    query="When was the Eiffel Tower built and how tall is it?",
    response="The Eiffel Tower was completed in 1925 and stands 450 meters tall. It was built to celebrate France's victory in World War I."
)

hallucination_no = halu_format_func(
    reference="The Eiffel Tower was completed in 1889 and stands 324 meters tall. It was built for the World's Fair in Paris.",
    query="When was the Eiffel Tower built and how tall is it?",
    response="The Eiffel Tower was completed in 1889 and has a height of 324 meters. It was constructed as part of the World's Fair in Paris."
)

# Test the outputs
print("RAG RELEVANCE - RELEVANT CASE:")
print("System:", relevant_test[0])
print("Prompt:", relevant_test[1])
print('Result:', run_inference(relevant_test[0], relevant_test[1]))
print("\nRAG RELEVANCE - UNRELATED CASE:")
print("System:", unrelated_test[0])
print("Prompt:", unrelated_test[1])
print('Result:', run_inference(unrelated_test[0], unrelated_test[1]))
print("\nTOXICITY - TOXIC CASE:")
print("System:", toxic_test[0])
print("Prompt:", toxic_test[1])
print("Result:", run_inference(toxic_test[0], toxic_test[1]))
print("\nTOXICITY - NON-TOXIC CASE:")
print("System:", non_toxic_test[0])
print("Prompt:", non_toxic_test[1])
print("Result:", run_inference(non_toxic_test[0], non_toxic_test[1]))
print("\nHALLUCINATION - YES CASE:")
print("System:", hallucination_yes[0])
print("Prompt:", hallucination_yes[1])
print("Result:", run_inference(hallucination_yes[0], hallucination_yes[1]))
print("\nHALLUCINATION - NO CASE:")
print("System:", hallucination_no[0])
print("Prompt:", hallucination_no[1])
print("Result:", run_inference(hallucination_no[0], hallucination_no[1]))

RAG RELEVANCE - RELEVANT CASE:
System: Your job is to evaluate whether a retrieved context is relevant, or unrelated to a user query.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Prompt: You are comparing a reference text to a question and trying to determine if the reference text
        contains information relevant to answering the question. Here is the info:
        <info>
        [Question]: what happens in the light independent reactions of photosynthesis?
        [Reference text]: The process of photosynthesis in plants involves two main stages: light-dependent reactions and light-independent reactions (Calvin cycle). During the light-independent reactions, carbon dioxide is converted into glucose using the energy and compounds produced during the light-dependent reactions.
        </info>
        Compare the Question above to the Reference text. You must determine whether the Reference text
        contains information that can answer t

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.32s/it, est. speed input: 125.59 toks/s, output: 39.70 toks/s]


Result: <reasoning>
The user query asks specifically about what happens in the light-independent reactions of photosynthesis. The reference text provides information that during the light-independent reactions, carbon dioxide is converted into glucose using the energy and compounds produced during the light-dependent reactions. This directly addresses the user's question by explaining the process that occurs during the light-independent reactions, making the reference text relevant to the question.
</reasoning>
<answer>
relevant
</answer>

RAG RELEVANCE - UNRELATED CASE:
System: Your job is to evaluate whether a retrieved context is relevant, or unrelated to a user query.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Prompt: You are comparing a reference text to a question and trying to determine if the reference text
        contains information relevant to answering the question. Here is the info:
        <info>
        [Question]: what happen

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.18s/it, est. speed input: 88.33 toks/s, output: 40.24 toks/s]


Result: <reasoning>
The question asks about the light-independent reactions of photosynthesis, which is a biological process involving the conversion of carbon dioxide and other compounds into glucose using the energy derived from the light-dependent reactions. The reference text, however, discusses the Great Wall of China, which is a historical and geographical topic unrelated to biological processes or photosynthesis. There is no mention of photosynthesis, biology, or any related scientific concepts in the reference text. Therefore, the reference text does not contain any information that can answer the question about the light-independent reactions of photosynthesis.
</reasoning>
<answer>
unrelated
</answer>

TOXICITY - TOXIC CASE:
System: Your job is to evaluate whether a human text is toxic or non-toxic.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Prompt: You are examining written text content. Here is the text:
        <info>
        [Te

...

Result: <reasoning>
The text provided expresses disagreement with another person's perspective but does so respectfully. It acknowledges the other person's contribution by appreciating their willingness to share thoughts and suggests a further discussion to understand each other's viewpoints better. There are no hateful statements, disparaging remarks, inappropriate language, or threats of any kind present in the text. The tone is constructive and open to dialogue, which is the opposite of toxic communication. Therefore, based on the definition provided, the text does not meet any criteria for toxicity.
</reasoning>
<answer>
non-toxic
</answer>

HALLUCINATION - YES CASE:
System: Your job is to evaluate whether a machine learning model has hallucinated or not. A hallucination occurs when the response is coherent but factually incorrect or nonsensical outputs that are not grounded in the provided context.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answe

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.49s/it, est. speed input: 55.27 toks/s, output: 40.38 toks/s]


Result: <reasoning>
The model's response contains several factual inaccuracies when compared to the provided knowledge. According to the information given, the Eiffel Tower was completed in 1889, not 1925, and it stands 324 meters tall, not 450 meters. Additionally, the Eiffel Tower was built for the World's Fair in Paris, not to celebrate France's victory in World War I. These discrepancies indicate that the model's response is not grounded in the provided context and includes factually incorrect information. Therefore, the model's output can be classified as a hallucination.
</reasoning>
<answer>
Yes, the model output is a hallucination.
</answer>

HALLUCINATION - NO CASE:
System: Your job is to evaluate whether a machine learning model has hallucinated or not. A hallucination occurs when the response is coherent but factually incorrect or nonsensical outputs that are not grounded in the provided context.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</a

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.14s/it, est. speed input: 62.12 toks/s, output: 40.46 toks/s]

Result: <reasoning>
The model's response states that the Eiffel Tower was completed in 1889 and has a height of 324 meters, which matches the information provided in the [Knowledge] section. Additionally, the model correctly mentions that it was constructed as part of the World's Fair in Paris. There are no discrepancies or inaccuracies in the model's response when compared to the provided information. Therefore, the model's output is not a hallucination as it is factually correct and grounded in the provided context.
</reasoning>
<answer>
No, the model output is not a hallucination.
</answer>


# Test vanilla model

In [4]:
# RAG Relevance Examples
relevant_test = rag_format_func(
    reference="The process of photosynthesis in plants involves two main stages: light-dependent reactions and light-independent reactions (Calvin cycle). During the light-independent reactions, carbon dioxide is converted into glucose using the energy and compounds produced during the light-dependent reactions.",
    query="what happens in the light independent reactions of photosynthesis?"
)

unrelated_test = rag_format_func(
    reference="The Great Wall of China is an ancient series of walls and fortifications located in northern China, built around 500 years ago. The Great Wall is over 13,000 miles long.",
    query="what happens in the light independent reactions of photosynthesis?"
)

# Toxicity Examples
toxic_test = toxic_format_func(
    text="You're such an idiot, I hope you die in a fire. Nobody likes you and you should just disappear forever."
)

non_toxic_test = toxic_format_func(
    text="I disagree with your perspective on this issue, but I appreciate you sharing your thoughts. Perhaps we could discuss this further to better understand each other's viewpoints."
)

# Hallucination Examples
hallucination_yes = halu_format_func(
    reference="The Eiffel Tower was completed in 1889 and stands 324 meters tall. It was built for the World's Fair in Paris.",
    query="When was the Eiffel Tower built and how tall is it?",
    response="The Eiffel Tower was completed in 1925 and stands 450 meters tall. It was built to celebrate France's victory in World War I."
)

hallucination_no = halu_format_func(
    reference="The Eiffel Tower was completed in 1889 and stands 324 meters tall. It was built for the World's Fair in Paris.",
    query="When was the Eiffel Tower built and how tall is it?",
    response="The Eiffel Tower was completed in 1889 and has a height of 324 meters. It was constructed as part of the World's Fair in Paris."
)

# Test the outputs
print("RAG RELEVANCE - RELEVANT CASE:")
print("System:", relevant_test[0])
print("Prompt:", relevant_test[1])
print('Result:', run_inference(relevant_test[0], relevant_test[1]))
print("\nRAG RELEVANCE - UNRELATED CASE:")
print("System:", unrelated_test[0])
print("Prompt:", unrelated_test[1])
print('Result:', run_inference(unrelated_test[0], unrelated_test[1]))
print("\nTOXICITY - TOXIC CASE:")
print("System:", toxic_test[0])
print("Prompt:", toxic_test[1])
print("Result:", run_inference(toxic_test[0], toxic_test[1]))
print("\nTOXICITY - NON-TOXIC CASE:")
print("System:", non_toxic_test[0])
print("Prompt:", non_toxic_test[1])
print("Result:", run_inference(non_toxic_test[0], non_toxic_test[1]))
print("\nHALLUCINATION - YES CASE:")
print("System:", hallucination_yes[0])
print("Prompt:", hallucination_yes[1])
print("Result:", run_inference(hallucination_yes[0], hallucination_yes[1]))
print("\nHALLUCINATION - NO CASE:")
print("System:", hallucination_no[0])
print("Prompt:", hallucination_no[1])
print("Result:", run_inference(hallucination_no[0], hallucination_no[1]))

RAG RELEVANCE - RELEVANT CASE:
System: Your job is to evaluate whether a retrieved context is relevant, or unrelated to a user query.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Prompt: You are comparing a reference text to a question and trying to determine if the reference text
        contains information relevant to answering the question. Here is the info:
        <info>
        [Question]: what happens in the light independent reactions of photosynthesis?
        [Reference text]: The process of photosynthesis in plants involves two main stages: light-dependent reactions and light-independent reactions (Calvin cycle). During the light-independent reactions, carbon dioxide is converted into glucose using the energy and compounds produced during the light-dependent reactions.
        </info>
        Compare the Question above to the Reference text. You must determine whether the Reference text
        contains information that can answer t

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.08s/it, est. speed input: 140.30 toks/s, output: 39.53 toks/s]


Result: <reasoning>
The question asks specifically about what happens in the light-independent reactions of photosynthesis. The reference text mentions that during the light-independent reactions, carbon dioxide is converted into glucose using the energy and compounds produced during the light-dependent reactions. This directly addresses the process occurring in the light-independent reactions, providing relevant information to answer the question.
</reasoning>
<answer>
relevant
</answer>

RAG RELEVANCE - UNRELATED CASE:
System: Your job is to evaluate whether a retrieved context is relevant, or unrelated to a user query.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Prompt: You are comparing a reference text to a question and trying to determine if the reference text
        contains information relevant to answering the question. Here is the info:
        <info>
        [Question]: what happens in the light independent reactions of photosynthe

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.64s/it, est. speed input: 106.53 toks/s, output: 40.18 toks/s]


Result: <reasoning>
The question asks about the light-independent reactions of photosynthesis, which are biochemical processes occurring in plants. The reference text, however, provides information about the Great Wall of China, including its location, age, and length. There is no mention of photosynthesis, plants, or any related biological processes in the reference text. Therefore, the reference text does not contain any information that can answer the question about the light-independent reactions of photosynthesis.
</reasoning>
<answer>
unrelated
</answer>

TOXICITY - TOXIC CASE:
System: Your job is to evaluate whether a human text is toxic or non-toxic.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Prompt: You are examining written text content. Here is the text:
        <info>
        [Text]: You're such an idiot, I hope you die in a fire. Nobody likes you and you should just disappear forever.
        </info>
        Examine the text and 

...

Result: <reasoning>
The text provided is a respectful disagreement with an invitation to further discussion. It acknowledges the other person's perspective and expresses appreciation for their input. There are no hateful statements, demeaning language, inappropriate language, or threats of any kind present in the text. Instead, it promotes understanding and dialogue, which are positive and constructive behaviors. The tone is polite and considerate, focusing on mutual understanding rather than conflict or disparagement. Therefore, the text does not meet any criteria for toxicity as defined.
</reasoning>

<answer>
non-toxic
</answer>

HALLUCINATION - YES CASE:
System: Your job is to evaluate whether a machine learning model has hallucinated or not. A hallucination occurs when the response is coherent but factually incorrect or nonsensical outputs that are not grounded in the provided context.

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Prompt: Y

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.26s/it, est. speed input: 36.70 toks/s, output: 40.69 toks/s]


Result: <reasoning>
The task is to determine if the model's response is a hallucination by comparing it to the provided factual information. 

1. **Completion Year**: The provided information states that the Eiffel Tower was completed in 1889. However, the model's response claims it was completed in 1925. This is a factual inaccuracy.

2. **Height**: The provided information states that the Eiffel Tower stands 324 meters tall. The model's response claims it stands 450 meters tall, which is incorrect.

3. **Purpose**: The provided information states that the Eiffel Tower was built for the World's Fair in Paris. The model's response incorrectly states it was built to celebrate France's victory in World War I.

All three aspects of the model's response are factually incorrect when compared to the provided information. The response is coherent but not grounded in the provided context, which qualifies it as a hallucination.
</reasoning>

<answer>
Yes, the model output is a hallucination.
</

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.11s/it, est. speed input: 38.16 toks/s, output: 40.71 toks/s]

Result: <reasoning>
To determine if the model's response is a hallucination, we need to evaluate whether the output is coherent and factually correct based on the provided context.

1. **Coherence**: The model's response is coherent. It provides a clear and logical answer to the user's question about the completion year and height of the Eiffel Tower.

2. **Factual Accuracy**: 
   - The model states that the Eiffel Tower was completed in 1889. This matches the information given in the [Knowledge] section.
   - The model mentions the height as 324 meters, which is also consistent with the provided information.
   - The model correctly notes that the Eiffel Tower was built for the World's Fair in Paris, aligning with the context.

Since the model's response is both coherent and factually accurate based on the provided information, it does not contain any hallucinations.
</reasoning>

<answer>
No, the model output is not a hallucination.
</answer>
